# Databases

## Using SQLite

## Query Language Syntax

In [ ]:
SELECT * FROM User 
    WHERE first_name = 'Alice'
    AND last_login > '2024-'

In [ ]:
ALTER TABLE User ADD COLUMN user_type_id INT;

In [ ]:
UPDATE User SET user_type_id = 3 WHERE first_name = 'Alice';
UPDATE User SET user_type_id = 2 WHERE first_name = 'Bob';
UPDATE User SET user_type_id = 2 WHERE first_name = 'Charlie';


SELECT 
    User.first_name, User.last_name, Usertype.name
FROM 
    User JOIN Usertype ON User.user_type_id = Usertype.user_type_id
WHERE
    User.email = 'a@example.com';

In [ ]:
SELECT
    u.first_name, u.last_name, ut.name
FROM
    User u JOIN Usertype ut ON u.user_type_id = ut.user_type_id
WHERE
    u.email = 'a@example.com';

In [ ]:
SELECT 
    u.first_name, u.last_name, ut.name
FROM 
    User u JOIN Usertype ut ON u.user_type_id = ut.user_type_id
WHERE
    u.email = 'a@example.com';

## sqlite3 connector

In [36]:
import sqlite3

conn = sqlite3.connect('unlocking-python.db')

In [37]:
cur = conn.cursor()

In [38]:
cur.execute('SELECT * FROM User')

In [39]:
cur.fetchone()

(1, 'Alice', 'Acevedo', 'alice123', 'a@example.com', '2024-01-01 12:00:00', 3)

In [40]:
cur.fetchall()

[(2, 'Bob', 'Brown', 'bobby_brown', 'b@example.com', '2024-02-01 12:00:00', 2),
 (3, 'Charlie', 'Clark', 'chuck', 'c@example.com', '2024-03-01 12:00:00', 2)]

In [41]:
cur.execute('SELECT * FROM User')
cur.fetchmany(2)

[(1,
  'Alice',
  'Acevedo',
  'alice123',
  'a@example.com',
  '2024-01-01 12:00:00',
  3),
 (2, 'Bob', 'Brown', 'bobby_brown', 'b@example.com', '2024-02-01 12:00:00', 2)]

In [42]:
cur.fetchmany(2)

[(3, 'Charlie', 'Clark', 'chuck', 'c@example.com', '2024-03-01 12:00:00', 2)]

In [43]:
cur.execute(
'''
INSERT INTO User (
first_name, last_name, username, email, last_login, user_type_id )
VALUES
('Daria', 'Dorgendorffer', 'daria', 'd@example.com', '2024-04-01 12:00:00', 1);
''' )


In [44]:
cur.execute('SELECT * FROM User')
cur.fetchall()

[(1,
  'Alice',
  'Acevedo',
  'alice123',
  'a@example.com',
  '2024-01-01 12:00:00',
  3),
 (2, 'Bob', 'Brown', 'bobby_brown', 'b@example.com', '2024-02-01 12:00:00', 2),
 (3, 'Charlie', 'Clark', 'chuck', 'c@example.com', '2024-03-01 12:00:00', 2),
 (5,
  'Daria',
  'Dorgendorffer',
  'daria',
  'd@example.com',
  '2024-04-01 12:00:00',
  1)]

In [45]:
conn.commit()

In [46]:
cur.close()

In [47]:
with sqlite3.connect('unlocking-python.db') as conn:
    cur = conn.cursor()
    res = cur.execute('SELECT * FROM User')
    print(res.fetchall())

[(1, 'Alice', 'Acevedo', 'alice123', 'a@example.com', '2024-01-01 12:00:00', 3), (2, 'Bob', 'Brown', 'bobby_brown', 'b@example.com', '2024-02-01 12:00:00', 2), (3, 'Charlie', 'Clark', 'chuck', 'c@example.com', '2024-03-01 12:00:00', 2), (5, 'Daria', 'Dorgendorffer', 'daria', 'd@example.com', '2024-04-01 12:00:00', 1)]


## Object Relational Mapping

In [48]:
class User:
    def __init__(self, data):
        pass

In [49]:
USER_SELECT_QUERY = '''
SELECT
    u.user_id,
    u.first_name,
    u.last_name,
    u.username,
    u.email,
    u.last_login,
    ut.name,
    ut.user_type_id,
    ut.monthly_cost
FROM 
    User u JOIN Usertype ut ON u.user_type_id = ut.user_type_id
'''
    

In [69]:
class User:
    def __init__(self, data):
        self.user_id = data[0]
        self.first_name = data[1]
        self.last_name = data[2]
        self.username = data[3]
        self.email = data[4]
        self.last_login = data[5]
        self.usertype_name = data[6]
        self.usertype_id = data[7]
        self.usertype_monthly_cost = data[8]

In [70]:
class SQLiteUserMapper:
    def __init__(self, conn):
        self.conn = conn

    def get_all_users(self):
        cur = self.conn.cursor()
        cur.execute(USER_SELECT_QUERY)
        return [User(d) for d in cur.fetchall()]
    
    def get_user_by_id(self, user_id):
        cur = self.conn.cursor()
        cur.execute(f'{USER_SELECT_QUERY} WHERE user_id = {user_id}')
        return User(cur.fetchone())


In [71]:
conn = sqlite3.connect('unlocking-python.db')
mapper = SQLiteUserMapper(conn)

In [72]:
user = mapper.get_user_by_id(1)
print(f'Got user {user.first_name} {user.last_name}')

Got user Alice Acevedo


In [73]:
users = mapper.get_all_users()
for u in users:
    print(f'Got user {u.first_name} {u.last_name}')

Got user Alice Acevedo
Got user Bob Brown
Got user Charlie Clark
Got user Daria Dorgendorffer


In [74]:

class SQLiteUserMapper:
    def __init__(self, conn):
        self.conn = conn

    def get_all_users(self):
        cur = self.conn.cursor()
        cur.execute(USER_SELECT_QUERY)
        return [User(d) for d in cur.fetchall()]
    
    def get_user_by_id(self, user_id):
        cur = self.conn.cursor()
        cur.execute(f'{USER_SELECT_QUERY} WHERE user_id = {user_id}')
        return User(cur.fetchone())

    def commit(self, u):
        cur = self.conn.cursor()
        update_query = f'''
        UPDATE User SET 
            first_name = '{u.first_name}',
            last_name = '{u.last_name}',
            username = '{u.username}',
            email = '{u.email}',
            last_login = '{u.last_login}',
            user_type_id = {u.user_type_id}
        WHERE user_id = {u.user_id}
        '''
        cur.execute(update_query)
        self.conn.commit()
        return self.get_user_by_id(u.user_id)

In [75]:
conn = sqlite3.connect('unlocking-python.db')
mapper = SQLiteUserMapper(conn)

bob = mapper.get_user_by_id(2)
print(f'{bob.first_name} has the {bob.usertype_name} membership')
bob.user_type_id = 3
bob = mapper.commit(bob)
print(f'{bob.first_name} has the {bob.usertype_name} membership')

User id is: 2
<built-in method fetchone of sqlite3.Cursor object at 0x16af95d40>
Bob has the free membership
User id is: 2
<built-in method fetchone of sqlite3.Cursor object at 0x16af95f40>
Bob has the premium membership


## Exercises

**1.**

Design two tables that hold social media post and comment data. Call these tables `Posts` and `Comments`. The `Comments` table should have a `post_id` (the ID of the post being commented on). Both tables should have a `user_id` column that points to the User table.

**2.**

Design a table that holds following/follower relationships between users. You should be able to write a query that gets a user’s followers, and also a query that gets a list of users they follow.


**3.**

Add a method to the `SQLiteUserMapper` that updates `last_login` to the current timestamp. This method should take only the `user` object as an argument and use the datetime package to get the current timestamp.

**4.**

Create a mapper class for `Usertype`, that allows us to make changes to the name and monthly costs of the `Usertype`.

**5.**

Rewrite the `User` and `SQLiteUserMapper` classes to use a `Usertype` class, rather than storing the user type data as flat attributes in the `User` class. That is, the `User` class should have an attribute, `self.usertype`, which points to a UserType object.